# Start points
Find the most northeasternly, or other orientation, point from a list of x,y locations.

In [1]:
from math import degrees, atan2, hypot
import fiona
import geopandas as gpd

In [2]:
in_data_path = r'../testdata/Sample_data.gdb'
layer_name = 'ROADS'

In [3]:
# define a record reader to avoid loading all of a layers attributes. This is purely to save memory, and won't 
# result in any speedup, but helps manage with large datasets.
# Credit to https://gis.stackexchange.com/questions/129414/only-read-specific-attribute-columns-of-a-shapefile-with-geopandas-fiona
def records(filename, usecols, **kwargs):
    with fiona.open(filename, **kwargs) as source:
        for feature in source:
            f = {k: feature[k] for k in ['id', 'geometry']}
            f['properties'] = {k: feature['properties'][k] for k in usecols}
            yield f

In [4]:
df = gpd.GeoDataFrame.from_features(records(in_data_path, usecols=['SHAPE_Length', 'NGD_UID'], layer=layer_name))
df.head()

,NGD_UID,SHAPE_Length,geometry
0,2758288,299.066820,"(LINESTRING (3722405.06 2709708.722857143, 372..."
1,2758636,569.803302,(LINESTRING (3723668.391428571 2710929.4457142...
2,2758874,277.067441,(LINESTRING (3727011.442857143 2714613.6571428...
3,2759085,525.544878,"(LINESTRING (3732677.502857143 2741944.84, 373..."
4,2758996,1033.104971,(LINESTRING (3734456.685714286 2745863.2114285...


In [5]:
df['start'] = df.geometry.apply(lambda x: x[0].coords[0])
df['end'] = df.geometry.apply(lambda x: x[0].coords[-1])
df.head()

,NGD_UID,SHAPE_Length,geometry,start,end
0,2758288,299.066820,"(LINESTRING (3722405.06 2709708.722857143, 372...","(3722405.06, 2709708.722857143)","(3722691.845714286, 2709793.5457142857)"
1,2758636,569.803302,(LINESTRING (3723668.391428571 2710929.4457142...,"(3723668.391428571, 2710929.4457142856)","(3724009.305714286, 2710557.0057142857)"
2,2758874,277.067441,(LINESTRING (3727011.442857143 2714613.6571428...,"(3727011.442857143, 2714613.657142857)","(3726924.334285714, 2714359.145714286)"
3,2759085,525.544878,"(LINESTRING (3732677.502857143 2741944.84, 373...","(3732677.502857143, 2741944.84)","(3732233.1685714284, 2741669.685714286)"
4,2758996,1033.104971,(LINESTRING (3734456.685714286 2745863.2114285...,"(3734456.685714286, 2745863.2114285715)","(3734517.92, 2744832.86)"


In [6]:
# naive centroid calculation - this is not guaranteed to generate an ST_Centroid() equivalent
# centroid is based purely on start points
centroid_y = max(df['start'][0]) - min(df['start'][0])
centroid_x = max(df['start'][1]) - min(df['start'][1])

In [7]:
df['bearing'] = df.start.apply(lambda p: round(degrees(atan2(p[0] - centroid_y, p[1] - centroid_x)), 4))
df['hypot'] = df.start.apply(lambda p: round(hypot(p[1] - centroid_x, p[0] - centroid_y),3))
df.head()

,NGD_UID,SHAPE_Length,geometry,start,end,bearing,hypot
0,2758288,299.066820,"(LINESTRING (3722405.06 2709708.722857143, 372...","(3722405.06, 2709708.722857143)","(3722691.845714286, 2709793.5457142857)",57.9429,3197221.886
1,2758636,569.803302,(LINESTRING (3723668.391428571 2710929.4457142...,"(3723668.391428571, 2710929.4457142856)","(3724009.305714286, 2710557.0057142857)",57.9364,3198940.520
2,2758874,277.067441,(LINESTRING (3727011.442857143 2714613.6571428...,"(3727011.442857143, 2714613.657142857)","(3726924.334285714, 2714359.145714286)",57.9123,3203729.706
3,2759085,525.544878,"(LINESTRING (3732677.502857143 2741944.84, 373...","(3732677.502857143, 2741944.84)","(3732233.1685714284, 2741669.685714286)",57.5542,3223111.938
4,2758996,1033.104971,(LINESTRING (3734456.685714286 2745863.2114285...,"(3734456.685714286, 2745863.2114285715)","(3734517.92, 2744832.86)",57.5124,3226716.460


In [8]:
def get_heading(bearing):
    headings = ("NE", "E", "SE", "S", "SW", "W", "NW", "N")
    index = bearing - 22.5
    if index < 0:
        index += 360
    index = int(index / 45)
    return headings[index]

In [9]:
df['heading'] = df.bearing.apply(lambda h: get_heading(h))
df.head()

,NGD_UID,SHAPE_Length,geometry,start,end,bearing,hypot,heading
0,2758288,299.066820,"(LINESTRING (3722405.06 2709708.722857143, 372...","(3722405.06, 2709708.722857143)","(3722691.845714286, 2709793.5457142857)",57.9429,3197221.886,NE
1,2758636,569.803302,(LINESTRING (3723668.391428571 2710929.4457142...,"(3723668.391428571, 2710929.4457142856)","(3724009.305714286, 2710557.0057142857)",57.9364,3198940.520,NE
2,2758874,277.067441,(LINESTRING (3727011.442857143 2714613.6571428...,"(3727011.442857143, 2714613.657142857)","(3726924.334285714, 2714359.145714286)",57.9123,3203729.706,NE
3,2759085,525.544878,"(LINESTRING (3732677.502857143 2741944.84, 373...","(3732677.502857143, 2741944.84)","(3732233.1685714284, 2741669.685714286)",57.5542,3223111.938,NE
4,2758996,1033.104971,(LINESTRING (3734456.685714286 2745863.2114285...,"(3734456.685714286, 2745863.2114285715)","(3734517.92, 2744832.86)",57.5124,3226716.460,NE
